문자열 토큰 처리 후 LSTM으로 감성분류(?)
이항분류하려고 하는데 긍정인가 부정인가를 분류하려고 함


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
import numpy as np
from keras.layers import SimpleRNN,LSTM,Dense


In [ ]:
samples=['The cat say on the mat.', 'The dog ate my homework']

token_index = {}
for sam in samples:
    for word in sam.split(sep=' '):
        if word not in token_index:
            token_index[word]=len(token_index)
print(token_index)
print()
tokenizer = Tokenizer(num_words=10)
tokenizer.fit_on_texts(samples)
token_seq = tokenizer.texts_to_sequences(samples)

print(token_seq)
print(tokenizer.word_index)
print()
token_mat=tokenizer.texts_to_matrix(samples, mode='binary') #'count','tdidf','feq'
print(token_mat) #원핫 이진벡터 형태
print(tokenizer.word_counts)
print(tokenizer.document_count)
print(tokenizer.word_docs)
print()

from keras.utils import to_categorical
token_seq = to_categorical(token_seq[0], num_classes=6)
print(token_seq)

In [ ]:
docs = ['너무 재밌네요','최고에요','참 잘 만든 영화에요','추천하고 싶은 영화입니다','한 번 더 보고 싶어요',
        '글쎄요','별로에요','생각보다 지루하네요','연기가 어색해요','재미없어요']
labels=np.array([1,1,1,1,1,0,0,0,0,0])

token=Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

x= token.texts_to_sequences(docs)
print('정수 인덱싱된 토큰 결과: ',x)

from keras.utils import pad_sequences #서로 다른 크기의 데이터를 일정한 크기로 만들어줌
from keras.layers import Embedding,Flatten
padded_x = pad_sequences(x, 5)
print(padded_x)

word_size= len(token.word_index)+1 # 임베딩에 입력될 단어수(토큰 수)를 지정 : 가능한 토큰 수는 최대값 +1을 준다.
model = Sequential()
# Embedding(가능토큰수, 벡터크기, input_length=시퀀스개수) - 워드 임베딩이란 텍스트 크기 단어를 밀집 벡터로 만드는 작업
# Embeddding은 워드 임베딩작업을 수행하고 3D 텐서를 리턴함
model.add(Embedding(word_size,8,input_length=5))#시퀀스 수 RNN 층
model.add(LSTM(units=32, activation='tanh')) #RNN층
# model.add(Flatten()) #빼고 그냥 해도 된다. label이기 때문에
model.add(Dense(units=32, activation='relu')) #Dense layer
model.add(Dense(units=1, activation='sigmoid')) #완전연결층
print(model.summary())

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(padded_x,labels,epochs=20,verbose=1)
print('eval : %.4f'%(model.evaluate(padded_x, labels)[1]))

print('predict : ', np.where(model.predict(padded_x)>0.5,1,0).ravel())
print('real : ', labels)

#lstm을 덴스 층에 올려놓고 이항분류를 해본것 참 거짓을 파악한 것

In [ ]:
# 시스템만 좋으면 됨 알고리즘만 좋으면 됨
# 위키 이런데 가면 데이터가 많다.

